Compute blue score given ground truth sequence along with predicted sequence

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from nltk.translate.bleu_score import sentence_bleu
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})


In [2]:
predictions = pd.read_csv("texnet_predictions.csv")


In [3]:
predictions

,eval_image_name,target_seq,pred_seq,score_pred,score_gold
0,0,\phi ( g ) = \lambda ( g ) \xi,\phi ( g ) = \lambda ( g ) \xi,0,0
1,12119,a _ { m 1 1 2 } = 0 .,a _ { m 1 1 2 } = 0 .,0,0
2,6769,v = x \left( \frac { N + 2 } { N } \right),v = x \left( \frac { N + 2 } { N } \right),0,0
3,6770,"\phi _ { j } ( \theta , \phi ) = g _ { j } ( \...","\phi _ { j } ( \theta , \phi ) = g _ { j } ( \...",0,0
4,12117,\hat { n } _ { \alpha \beta },\hat { n } _ { \alpha \beta },0,0
...,...,...,...,...,...
16515,12372,"\frac 1 2 , \frac 1 2 , \frac 1 2","\frac { 1 } { 2 } , \frac { 1 } { 2 } , \frac ...",0,0
16516,15630,\tau ^ { \mathbf { \hat { a } } },{ { \mathrm { \boldmath { \tau } } } ^ { { \bf...,0,0
16517,3860,"\left\langle \cdot , \cdot \right\rangle","{ \langle { \cdot , \cdot } \rangle }",0,0
16518,3218,| 0 _ { \mathrm { i n } } \rangle,{ \mathrm { { | } } { { { \mathrm { { 0 } } _ ...,0,0


In [4]:
split = lambda s:  s.split()
bleu_string = lambda truth, candidatate: sentence_bleu([split(truth)], split(candidatate), weights=(.50,.50,0,0))

In [5]:
predictions["score_pred"] = predictions.apply(lambda r: bleu_string(r["target_seq"], r["pred_seq"]), axis=1)


In [43]:
bleu_string("a "*32, "a "*31+"b")

0.9682458365518543

In [38]:
predictions["length"] = predictions.target_seq.apply(split).apply(len)
predictions["score_pred"].describe()


0        10
1        11
2        14
3        27
4         9
         ..
16515    11
16516    11
16517     5
16518    11
16519     4
Name: length, Length: 16520, dtype: int64

In [7]:
hist = predictions.hist(bins=50, column=["length"])
fig = plt.figure()
fig.suptitle('Distribution of Lengths', fontsize=16)
plt.xlabel('length in tokens', fontsize=14)
plt.ylabel('frequency', fontsize=14)
plt.hist(predictions["length"], bins=50, label=None)
fig.patch.set_visible(False)
plt.savefig("assets/histogram.pdf")

In [25]:
fig = plt.figure()
fig.suptitle('Distribution of Scores', fontsize=16)
plt.xlabel('length in tokens', fontsize=14)
plt.ylabel('frequency', fontsize=14)
plt.hist(predictions["score_pred"], bins=100, label=None)
fig.patch.set_visible(False)
plt.savefig("assets/scorehistogram.pdf")

https://mode.com/example-gallery/python_histogram/
for the qualitative plots
https://stackoverflow.com/questions/6871201/plot-two-histograms-on-single-chart-with-matplotlib


In [40]:
score_by_len = predictions[["length","score_pred"]]
score_by_len["length"] = score_by_len["length"] // 5 * 5
splot = score_by_len.groupby("length").agg("mean")

fig = plt.figure()
fig.suptitle('Performance by Length', fontsize=16)
plt.xlabel('length in tokens', fontsize=14)
plt.ylabel('Bigram BLEU Score', fontsize=14)
plt.plot(splot)

plt.savefig('assets/scorebylen.pdf')


,score_pred
length,
0,0.853270
5,0.882014
10,0.921093
15,0.932694
20,0.942030
25,0.937890
30,0.937400
35,0.929381
40,0.927706


In [39]:
p = predictions[["target_seq","length","score_pred"]]
p["length"] = p["length"] // 2 * 2
has = lambda x: p["target_seq"].str.contains(x)
stacked = p[has("array") | has("cases") | has("matrix") | has("binom")]
splot = stacked.groupby("length").agg("mean")
fig = plt.figure()
fig.suptitle('Performance on Matrices', fontsize=16)
plt.xlabel('length in tokens', fontsize=14)
plt.ylabel('Bigram BLEU Score', fontsize=14)
plt.plot(splot)

plt.savefig('assets/scorebylenstacked.pdf')
stacked["score_pred"].count()


291

In [35]:
stacked = p[has("frac") | has("over")]
stacked = stacked[["length","score_pred"]]
splot = stacked.groupby("length").agg("mean")
fig = plt.figure()
fig.suptitle('Performance on Fractions', fontsize=16)
plt.xlabel('length in tokens', fontsize=14)
plt.ylabel('Bigram BLEU Score', fontsize=14)
plt.plot(splot)
plt.savefig('assets/scorebyfrac.pdf')
stacked["score_pred"].count()


2897